<a href="https://colab.research.google.com/github/aprakash596/Leveraging-Machine-Learning-to-Predict-Solar-Activity/blob/main/SolarActivityModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

# Upload the CSV file
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]

# Read the CSV file into a DataFrame
df = pd.read_csv(filename)

# Display the first few rows
df.head()

print(len(df))

Saving data.csv to data (1).csv
1389


In [ ]:
X = df[['previous 24 hour flare activity','evolution','common flares','area','common flares','moderate flares','severe flares','historically-complex']].values.tolist()
Y = df[['activity']].values.tolist()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
model = RandomForestClassifier(max_depth=2).fit(X_train,Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
predictions = model.predict(X_test)

In [ ]:
rForest2 = RandomForestClassifier(max_depth=1).fit(X_train,Y_train)
rForest3 = RandomForestClassifier(max_depth=100).fit(X_train,Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
predictions_randomForest = rForest2.predict(X_test)
predictions_randomForest2 = rForest3.predict(X_test)

can sum up the accuracies, if the accuracies are a1, a2, a3... then the sum will be a1 + a2 + a3

the weight of the first model is equal to a1/sum
the same process would be applied for the other models
the weight of the second model would be a2/sum

if don't know the accuracy, will look at how some of the models fail the first case and then take the average, then see the first model failed while the rest succeeded, and then manually reduce the weighting of the first model

everytime a model performs better, then have its weight increased, vice versa; if a lot are working well, then average will go over 1 so use sum

not okay to have a negative weight, make sure to always keep it positive



In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

below is a neural network. input layer is all of the parameters or values. the layers are numbers that change the inputs, and then the outputs are added together

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=200).fit(X_train, Y_train)
predictions_MLP = clf.predict(X_test)
print(accuracy_score(Y_test, predictions_MLP)," MLPClassifier test accuracy")
predictionsTrain_MLP = clf.predict(X_train)
print(accuracy_score(Y_train, predictionsTrain_MLP)," MLPClassifier train accuracy")

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8976034858387799  MLPClassifier test accuracy
0.8870967741935484  MLPClassifier train accuracy


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


randomForest is multiple decision trees, at the bottom, take the average of all of the outputs

In [ ]:
rForest2 = RandomForestClassifier(max_depth=20).fit(X_train,Y_train)
predictions_randomForest = rForest2.predict(X_test)
print(accuracy_score(Y_test, predictions_randomForest), " randomForest test accuracy")
predictionsTrain_randomForest = rForest2.predict(X_train)
print(accuracy_score(Y_train, predictionsTrain_randomForest), " randomForest train accuracy")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8888888888888888  randomForest test accuracy
0.896774193548387  randomForest train accuracy


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

when finding the k point, try to find the k points that are more similar to this. then just take the majority vote of these points (n_neighors = 3 means taking the majority of 3 points)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10).fit(X_train,Y_train)
predictions_Neighbor = neigh.predict(X_test)
print(accuracy_score(Y_test,predictions_Neighbor)," kNeighborsClassifier test accuracy")
predictionsTrain_Neighbor = neigh.predict(X_train)
print(accuracy_score(Y_train,predictionsTrain_Neighbor)," kNeighborsClassifier test accuracy")

0.8954248366013072  kNeighborsClassifier test accuracy
0.8763440860215054  kNeighborsClassifier test accuracy


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
from sklearn import tree

In [ ]:
tree_model = tree.DecisionTreeClassifier(max_depth=7)
tree_model = tree_model.fit(X_train, Y_train)
predictionsTrain_tree = tree_model.predict(X_train)
print(accuracy_score(Y_train,predictionsTrain_tree)," Decision tree train accuracy")
predictions_tree = tree_model.predict(X_test)
print(accuracy_score(Y_test,predictions_tree)," Decision tree test accuracy")

0.8913978494623656  Decision tree train accuracy
0.8845315904139434  Decision tree test accuracy


now will combine the models

In [ ]:
import numpy as np
from collections import Counter
import random


# Stack lists to iterate over each index
predictions = np.array([predictions_MLP, predictions_randomForest, predictions_Neighbor, predictions_tree]).T

# Compute majority vote with random choice in case of a tie
majority_vote = []
for votes in predictions:
    count = Counter(votes)
    max_count = max(count.values())
    candidates = [key for key, value in count.items() if value == max_count]
    majority_vote.append(random.choice(candidates))

print(accuracy_score(Y_test,majority_vote))

0.8910675381263616


write some paragraphs on what each model does and try different parameters with each model to get a better individual accuracy. create a table for each change in the model and the training and test accuracy

https://www.lumiere-education.com/post/15-journals-to-publish-your-research-in-high-school